# Setup

Import the relevant packages:

In [1]:
import micropip
await micropip.install("https://opencompl.github.io/tablegen-stats/files/analyze_tablegen-0.0.1-py3-none-any.whl")
from analyze_tablegen.main import *
await micropip.install("ipywidgets")
import ipywidgets as widgets
import matplotlib.pyplot as plt

Get the data from TableGen:

In [2]:
import json
from js import fetch
    
res = await fetch('https://opencompl.github.io/tablegen-stats/files/tablegen_data.json')
text = await res.text()
    
stats = get_stats_from_json(text)
dialects = stats.dialects
dialect_names = dialects.keys()

A simple bar plotter:

In [3]:
def plot_simple_consecutive_integer_data(values):
    keys = [str(i) for i in range(len(values))]
    fig = plt.bar(keys, values)
    plt.show()
    plt.close()
    
def category_plot(distribution):
    category_select = widgets.ToggleButtons(
        options=distribution.keys(),
        description='Category:',
        disabled=False,
        button_style='',
    )

    @widgets.interact(category=category_select)
    def category_plot_(category):
        all_distribution = distribution[category][0]
        per_dialect_distribution = distribution[category][1]

        plot_simple_consecutive_integer_data(all_distribution)

        toggle_select =widgets.ToggleButtons(
            options=dialect_names,
            description='Dialect:',
            disabled=False,
            button_style='', # 'success', 'info', 'warning', 'danger' or ''
        )

        @widgets.interact(dialect_name=toggle_select)
        def plot_dialect(dialect_name):
            plot_simple_consecutive_integer_data(per_dialect_distribution[dialect_name])

# Some general statistics
## General statistics on operations

In [4]:
distribution = dict()
distribution["operands"] = get_global_op_distribution(stats, lambda x: x.numOperands)
distribution["results"] = get_global_op_distribution(stats, lambda x: x.numResults)
distribution["regions"] = get_global_op_distribution(stats, lambda x: x.numRegions)
distribution["attributes"] = get_global_op_distribution(stats, lambda x: len(x.attributes))
distribution["C++ printer"] = get_global_op_distribution(stats, lambda x: 1 - int(x.hasAssemblyFormat))
distribution["C++ verifier"] = get_global_op_distribution(stats, lambda x: int(x.hasVerifier))
distribution["traits"] = get_global_op_distribution(stats, lambda x: len(x.traits))
distribution["interfaces"] = get_global_op_distribution(stats, lambda x: len(x.interfaces))

category_plot(distribution)

interactive(children=(ToggleButtons(description='Category:', options=('operands', 'results', 'regions', 'attri…

## General statistics on types

In [5]:
distribution = dict()
distribution["parameters"] = get_global_type_distribution(stats, lambda x: len(x.parameters))
distribution["traits"] = get_global_type_distribution(stats, lambda x: len(x.traits))
distribution["interfaces"] = get_global_type_distribution(stats, lambda x: len(x.interfaces))

category_plot(distribution)

interactive(children=(ToggleButtons(description='Category:', options=('parameters', 'traits', 'interfaces'), v…

## General statistics on attributes

In [6]:
distribution = dict()
distribution["parameters"] = get_global_attr_distribution(stats, lambda x: len(x.parameters))
distribution["traits"] = get_global_attr_distribution(stats, lambda x: len(x.traits))
distribution["interfaces"] = get_global_attr_distribution(stats, lambda x: len(x.interfaces))

category_plot(distribution)

interactive(children=(ToggleButtons(description='Category:', options=('parameters', 'traits', 'interfaces'), v…

# "Declarativeness" of current dialects

In [7]:
distribution = dict()
# TODO check constraints in types/attributes
distribution["types"] = get_global_type_distribution(stats, lambda x: int(not x.hasVerifier))
distribution["attributes"] = get_global_attr_distribution(stats, lambda x: int(not x.hasVerifier))
distribution["op has C++ parser"] = get_global_op_distribution(stats, lambda x: int(not x.hasAssemblyFormat))
distribution["op has C++ verifier"] = get_global_op_distribution(stats, lambda x: int(x.hasVerifier))
# Number of operations that have declarative operands/results
distribution["decl operands/results"] = get_global_op_distribution(stats, lambda x: int(x.is_operands_results_attrs_declarative()))
# Number of operations that have declarative traits
distribution["decl op traits"] = get_global_op_distribution(stats, lambda x: int(x.is_traits_declarative()))
# Number of operations that have declarative operands, results, and no C++ verifiers
distribution["decl ops without traits/interfaces"] = get_global_op_distribution(stats, lambda x: int(x.is_declarative(check_traits=False, check_interfaces=False)))
# Fully declarative
distribution["decl ops without interfaces"] = get_global_op_distribution(stats, lambda x: int(x.is_declarative(check_traits=True, check_interfaces=False)))
# Fully declarative
distribution["decl ops"] = get_global_op_distribution(stats, lambda x: int(x.is_declarative(check_traits=True, check_interfaces=True)))

category_plot(distribution)

interactive(children=(ToggleButtons(description='Category:', options=('types', 'attributes', 'op has C++ parse…

# Printing dialects in IRDL

In [8]:
import analyze_tablegen.simplifier as simplifier

stats_optimized = simplifier.simplify(stats)

constr = stats.dialects["tosa"].ops["tosa.apply_scale"].operands[2].constraint
constr_simplified = stats_optimized.dialects["tosa"].ops["tosa.apply_scale"].operands[2].constraint
print(constr)
print(constr_simplified)
print(simplifier.AndRedundantSimplifier.is_redundant(constr_simplified.operands[1], constr_simplified.operands[2]))
print(simplifier.AndRedundantSimplifier.simplify(constr_simplified))

category_select = widgets.ToggleButtons(
    options=dialect_names,
    description='Dialect:',
    disabled=False,
    button_style='',
)

@widgets.interact(dialect_name=category_select)
def print_dialect(dialect_name):
    
    optimization_select = widgets.ToggleButtons(
        options=["unoptimized", "optimized"],
        description='',
        disabled=False,
        button_style='',
    )
    
    @widgets.interact(optimization=optimization_select)
    def print_optimized(optimization):
        if optimization == "optimized":
            stats_optimized.dialects[dialect_name].print()
        else:
            stats.dialects[dialect_name].print()

AnyOf<builtin.integer<8, "Signless">, AnyOf<And<CppClass<::mlir::VectorType>, ShapedTypeOf<builtin.integer<8, "Signless">>>, And<CppClass<::mlir::TensorType>, ShapedTypeOf<builtin.integer<8, "Signless">>>>>
AnyOf<builtin.integer<8, "Signless">, builtin.vector<Any, builtin.integer<8, "Signless">>, builtin.tensor<Any, builtin.integer<8, "Signless">, Any>>
None
None


interactive(children=(ToggleButtons(description='Dialect:', options=('builtin', 'sparse_tensor', 'tosa', 'scf'…